## Job Web-Scraping
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [692]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en import STOP_WORDS

import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import coherencemodel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [693]:
#this is an input for jobs and
occupation = 'Data analyst' #str(input("Job title: "))
loc =  'Austin, TX' #str(input("City, State(i.e. TX): "))

In [694]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [695]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [696]:
#adds the data in URL format by removing any white space and adding the data. 
def Keyword_to_url(kw):
    kw = kw.strip(" ").replace(" ","%20")
    return kw

In [697]:
# this will be used as the URL input for multiple job searches. 
radius = str("5")
page_sort = "sortcolumns=accquisitiondate&sortdirections=DSC"
page_size = "100" #how many pages would you like to get. 
source = 'IN' #stands for indeed

In [698]:
#give it the siteD
url = 'https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword='+ \
    Keyword_to_url(occupation)+\
    '&ajax=0&location='+loc+'&radius='+radius+ \
    '&source='+ source +'&pagesize='+page_size+'&sortcolumns=accquisitiondate&sortdirections=DSC'
browser.get(url)
time.sleep(1)

In [699]:
#these catch the title of the job details. 
company = []
position = []
location = []
date_posted = []
job_url = []
pos_description = []

In [700]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-title="Job Title"]'))

In [701]:
job_links = browser.find_elements_by_xpath('//*[@data-title="Job Title"]')

In [704]:
def collect_job_listings(num):
    
    """
    Works on an individual link basis.
    Creates dataframe from the link and job listed on the site. 
    the job descriptions will be added later.
    """
    
    num = int(num)
    position.append(job_links[num].text)
    company.append(browser.find_elements_by_xpath('//*[@data-title="Company"]')[num].text)
    location.append(browser.find_elements_by_xpath('//*[@data-title="Location"]')[num].text)
    date_posted.append(browser.find_elements_by_xpath('//*[@data-title="Date Posted"]')[num].text)
    job_url.append(job_links[num].find_element_by_tag_name("a").get_attribute('href'))
    
    return pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},columns=['company','location','position','date_posted','position_description'])
    

In [705]:
#loop to obtain all information for each job in the table.
for links in range(listing_num):
    collect_job_listings(links)

In [706]:
#creating a dataframe of all data collected.
job_list = pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},columns=['company','location','position','date_posted','position_description']).drop_duplicates()
#drops all duplicates and replaces them with

In [707]:
#checks for page errors and to see which ones. 
err = []

In [708]:
#Loops the urls and receives the content from the webpage.
for i in tqdm(range(job_list.shape[0])):
    time.sleep(np.random.randint(2,7))
    try:
        browser.get(job_listing['job_url'][i])
        pos_des = browser.find_element_by_class_name("snip").text
        pos_des = pos_des.replace("\n",' ')
        job_list['position_description'][i] = job_list['position_description'][i].replace('0',pos_des)
    except:
        err.append(i)
        

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:20<00:00,  5.00s/it]


In [709]:
job_list.head()

,company,date_posted,job_url,location,position,position_description
0,Texas Commission on Environmental Quality,06/22/2018,http://www.indeed.com/viewjob?jk=74d1c3f5236b5...,"Austin, TX",Senior IT Business Analyst (Business Analyst III),The Recovery Data Analyst is responsible for p...
1,Storm Global Analytics,06/22/2018,http://www.indeed.com/viewjob?jk=d12ce849117df...,"Austin, TX",Digital Analytics Risk Analyst/Modeler,Position: Business Data Analyst - 629375 Loca...
2,Charles Schwab,06/21/2018,http://www.indeed.com/viewjob?jk=83ea07e9089cf...,"Austin, TX",Operations Specialist,--------------- About the Role: -------------...
3,Cray,06/21/2018,http://www.indeed.com/viewjob?jk=07c28805d4120...,"Austin, TX",Business Analyst - Product Strategy,Today more than 200 million people come to Pin...
4,EXL,06/21/2018,http://www.indeed.com/viewjob?jk=3e8d13ac7cc3a...,"Austin, TX","Digital Analytics Consultant, Decision Analytics",Description Chase is undertaking an aggressiv...


In [710]:
browser.close()

In [711]:
job_list.to_csv("./data/"+str(datetime.datetime.today())[:10]+"_"+occupation+"_"+loc+" job_list.csv")

In [733]:
here = []
for i in range(job_list.shape[0]):
    if job_list['position_description'][i].find('datum') > -1:
        here.append(i)
        

In [734]:
here

[]

## Using SpaCy for NLP - this is a Work in Progress
Objective: Setup for topic modeling and use LDA to determine feature importance

In [921]:
nlp = spacy.load('en')

In [922]:
pos_description = job_list['position_description'].values

In [923]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(pos_description),deacc=True))

In [924]:
data_words = list(sent_to_words(pos_description))

In [925]:
#Building bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=4, threshold=100)

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [926]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [927]:
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOP_WORDS] for doc in text]

def make_bigrams(text):
    return [bigram_mod[doc] for doc in text]

def make_trigrams(text):
    return [trigram_mod[bigram_mod[doc]] for doc in text]

def lemmatization(text, allowed_postags = ['NOUN','ADJ','VERB','ADV']):
    text_rem = []
    for sent in text:
        doc = nlp(" ".join(sent))
        text_rem.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return text_rem 

In [929]:
remove_more_words = ['skill','datum','day','week','year','ability','analyst','recovery','austin','document']

In [928]:
words_no_stops = remove_stopwords(data_words)

word_bigrams = make_bigrams(words_no_stops)

data_lemma =  lemmatization(word_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [943]:
data_lemma = [word for word in data_lemma[0] if word not in remove_more_words]

In [944]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,num_topics=10,
                               random_state=42,
                               update_every=1,
                               chunksize=100,
                               passes=10,
                               alpha='auto',
                               per_word_topics=False)

In [945]:
doc_lda = lda_model[corpus]

In [946]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = coherencemodel.CoherenceModel(model=lda_model, texts=data_lemma[0] , dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.2282796383510926

Coherence Score:  nan


In [947]:
pyLDAvis.gensim.prepare(lda_model,corpus, id2word)

PreparedData(topic_coordinates=            Freq  cluster  topics         x             y
topic                                                    
5      99.999741        1       1  0.173659 -7.052707e-09
4       0.000034        1       2 -0.031396  3.948146e-05
6       0.000033        1       3 -0.029508 -8.530109e-05
0       0.000031        1       4 -0.024353  5.354967e-05
3       0.000031        1       5 -0.023843  2.775122e-05
8       0.000030        1       6 -0.020450 -2.404567e-05
2       0.000029        1       7 -0.019665 -2.249622e-05
7       0.000029        1       8 -0.014197  7.911242e-05
1       0.000028        1       9 -0.008261 -1.184842e-04
9       0.000028        1      10 -0.001987  5.043949e-05, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
548   Default  808.000000        datum  808.000000  30.0000  30.0000
683   Default  628.000000      analyst  628.000000  29.0000  29.0000
727   Default  439.000000     business  439.000000  28.0000  28.0000
3409  Default  440.000000   experience  440.000000  27.0000  27.0000
1691  Default  412.000000          day  412.000000  26.0000  26.0000
3397  Default  367.000000         data  367.000000  25.0000  25.0000
2187  Default  240.000000      company  240.000000  24.0000  24.0000
2590  Default  231.000000      product  231.000000  23.0000  23.0000
129   Default  237.000000     analytic  237.000000  22.0000  22.0000
2197  Default  221.000000          job  221.000000  21.0000  21.0000
2798  Default  192.000000     analysis  192.000000  20.0000  20.0000
3354  Default  185.000000    marketing  185.000000  19.0000  19.0000
1842  Default  169.000000        skill  169.000000  18.0000  18.0000
2717  Default  152.000000        drive  152.000000  17.0000  17.0000
1607  Default  150.000000      insight  150.000000  16.0000  16.0000
333   Default  144.000000      ability  144.000000  15.0000  15.0000
104   Default  145.000000       easily  145.000000  14.0000  14.0000
2612  Default  140.000000         tool  140.000000  13.0000  13.0000
957   Default  138.000000         help  138.000000  12.0000  12.0000
28    Default  136.000000      support  136.000000  11.0000  11.0000
1328  Default  131.000000      project  131.000000  10.0000  10.0000
1266  Default  125.000000        build  125.000000   9.0000   9.0000
2216  Default  124.000000  opportunity  124.000000   8.0000   8.0000
423   Default  120.000000   management  120.000000   7.0000   7.0000
38    Default  119.000000       report  119.000000   6.0000   6.0000
2884  Default  121.000000         work  121.000000   5.0000   5.0000
1121  Default  113.000000      process  113.000000   4.0000   4.0000
1976  Default  111.000000          sql  111.000000   3.0000   3.0000
1504  Default  111.000000      provide  111.000000   2.0000   2.0000
1207  Default  112.000000          new  112.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
2187  Topic10    0.000013      company  240.272400  -1.6569  -6.6116
3409  Topic10    0.000015   experience  440.321869  -2.1084  -6.4574
1691  Topic10    0.000014          day  412.503540  -2.0949  -6.5092
3397  Topic10    0.000013         data  367.174042  -2.0768  -6.6075
2590  Topic10    0.000011      product  231.442245  -1.7622  -6.7544
129   Topic10    0.000011     analytic  237.545883  -1.8237  -6.7898
423   Topic10    0.000008   management  120.703537  -1.4754  -7.1186
1976  Topic10    0.000007          sql  111.802170  -1.4203  -7.1401
957   Topic10    0.000008         help  138.522430  -1.5958  -7.1013
2798  Topic10    0.000008     analysis  192.926697  -1.8740  -7.0482
1266  Topic10    0.000007        build  125.624260  -1.5799  -7.1831
333   Topic10    0.000007      ability  144.531693  -1.7125  -7.1755
2598  Topic10    0.000007    operation   97.942505  -1.4253  -7.2774
1121  Topic10    0.000007      process  113.825165  -1.5434  -7.2452
1336  Topic10    0